without softmax

In [1]:
import random
import math
from tabulate import tabulate

class Instance:
    def __init__(self, id, content):
        self.id = id
        self.content = content
        self.priority = 0.5
        self.confidence = 0.5
        self.use_count = 0

class FewShotSystem:
    def __init__(self, num_instances=20, instances_to_select=5, similarity_threshold=0.5, epsilon=0.2):
        self.instances = [Instance(i, f"Content_{i}") for i in range(num_instances)]
        self.instances_to_select = instances_to_select
        self.similarity_threshold = similarity_threshold
        self.epsilon = epsilon
        self.alpha = 0.1
        self.beta = 0.1
        self.delta_p_positive = 0.05
        self.delta_p_negative = 0.05

    def simulate_similarity(self, task, instance):
        return random.random()

    def select_instances(self, task):
        similar_instances = [inst for inst in self.instances if self.simulate_similarity(task, inst) > self.similarity_threshold]
        if len(similar_instances) < self.instances_to_select:
            similar_instances = self.instances

        if random.random() < self.epsilon:
            return random.sample(similar_instances, self.instances_to_select)
        else:
            sorted_instances = sorted(similar_instances, key=lambda x: x.priority, reverse=True)
            return sorted_instances[:self.instances_to_select]

    def update_priority_and_confidence(self, instance, is_useful):
        if is_useful:
            delta_p = self.delta_p_positive * math.exp(-self.alpha * instance.use_count)
            instance.priority = min(1.0, instance.priority + delta_p)
            instance.confidence = min(1.0, instance.confidence + 0.1)
        else:
            delta_p = self.delta_p_negative * math.exp(-self.beta * instance.use_count)
            instance.priority = max(0.1, instance.priority - delta_p)
            instance.confidence = max(0.1, instance.confidence - 0.1)
        instance.use_count += 1

    def weighted_decision(self, selected_instances):
        total_weight = sum(inst.priority * inst.confidence for inst in selected_instances)
        decision = sum(inst.priority * inst.confidence * inst.id for inst in selected_instances) / total_weight if total_weight != 0 else 0
        return decision

    def run_iteration(self, task):
        selected = self.select_instances(task)
        is_useful = random.choice([True, False])

        initial_states = {inst.id: (inst.priority, inst.confidence) for inst in self.instances}

        for inst in selected:
            self.update_priority_and_confidence(inst, is_useful)

        results = []
        for inst in self.instances:
            initial_priority, initial_confidence = initial_states[inst.id]
            if inst in selected:
                results.append([
                    inst.id,
                    initial_priority,
                    initial_confidence,
                    is_useful,
                    inst.priority,
                    inst.confidence
                ])
            else:
                results.append([
                    inst.id,
                    initial_priority,
                    initial_confidence,
                    None,
                    initial_priority,
                    initial_confidence
                ])

        decision = self.weighted_decision(selected)
        return selected, is_useful, results, decision

    def run_simulation(self, num_iterations=20):
        all_results = []
        for i in range(num_iterations):
            task = f"Task_{i}"
            selected, is_useful, results, decision = self.run_iteration(task)
            selected_ids = [inst.id for inst in selected]
            print(f"\nIteration {i+1}")
            print(f"Task: {task}")
            print(f"Selected instances: {selected_ids}")
            print(f"Is useful: {is_useful}")
            print(f"Weighted decision: {decision:.4f}")
            table = tabulate(results, headers=["ID", "Initial Priority", "Initial Confidence",
                                               "Is Useful", "Updated Priority", "Updated Confidence"],
                             floatfmt=".4f", tablefmt="grid")
            print(table)
            all_results.append(results)
        return all_results

system = FewShotSystem()
simulation_results = system.run_simulation()


Iteration 1
Task: Task_0
Selected instances: [2, 10, 11, 12, 13]
Is useful: False
Weighted decision: 9.6000
+------+--------------------+----------------------+-------------+--------------------+----------------------+
|   ID |   Initial Priority |   Initial Confidence | Is Useful   |   Updated Priority |   Updated Confidence |
+======+====================+======================+=============+====================+======================+
|    0 |             0.5000 |               0.5000 |             |             0.5000 |               0.5000 |
+------+--------------------+----------------------+-------------+--------------------+----------------------+
|    1 |             0.5000 |               0.5000 |             |             0.5000 |               0.5000 |
+------+--------------------+----------------------+-------------+--------------------+----------------------+
|    2 |             0.5000 |               0.5000 | False       |             0.4500 |               0.4000 |
+--

Confidence

In [5]:
import random
import math
import numpy as np
from tabulate import tabulate

class Instance:
    def __init__(self, id, content):
        self.id = id
        self.content = content
        self.priority = 0.5  # Initial priority
        self.confidence = 0.5  # Initial confidence
        self.use_count = 0

class FewShotSystem:
    def __init__(self, num_instances=20, instances_to_select=5, similarity_threshold=0.5, epsilon=0.2):
        self.instances = [Instance(i, f"Content_{i}") for i in range(num_instances)]
        self.instances_to_select = instances_to_select
        self.similarity_threshold = similarity_threshold
        self.epsilon = epsilon
        self.alpha = 0.1  # Attenuation parameters for positive feedback
        self.beta = 0.1  # Negative feedback attenuation parameter
        self.delta_p_positive = 0.05  # Initial positive adjustment
        self.delta_p_negative = 0.05  # initial negative adjustment amount

    def simulate_similarity(self, task, instance):
        return random.random()

    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def select_instances(self, task):
        similar_instances = [inst for inst in self.instances if self.simulate_similarity(task, inst) > self.similarity_threshold]
        if len(similar_instances) < self.instances_to_select:
            similar_instances = self.instances

        if random.random() < self.epsilon:
            return random.sample(similar_instances, self.instances_to_select)
        else:
            priorities = np.array([inst.priority for inst in similar_instances])
            normalized_priorities = self.softmax(priorities)
            sorted_instances = sorted(zip(similar_instances, normalized_priorities), key=lambda x: x[1], reverse=True)
            top_instances = [inst[0] for inst in sorted_instances[:self.instances_to_select]]
            return top_instances

    def update_priority_and_confidence(self, instance, is_useful):
        if is_useful:
            delta_p = self.delta_p_positive * math.exp(-self.alpha * instance.use_count)
            instance.priority = min(1.0, instance.priority + delta_p)
            instance.confidence = min(1.0, instance.confidence + 0.1)
        else:
            delta_p = self.delta_p_negative * math.exp(-self.beta * instance.use_count)
            instance.priority = max(0.1, instance.priority - delta_p)
            instance.confidence = max(0.1, instance.confidence - 0.1)
        instance.use_count += 1

    def weighted_decision(self, selected_instances):
        total_weight = sum(inst.priority * inst.confidence for inst in selected_instances)
        decision = sum(inst.priority * inst.confidence * inst.id for inst in selected_instances) / total_weight if total_weight != 0 else 0
        return decision

    def run_iteration(self, task):
        selected = self.select_instances(task)
        is_useful = random.choice([True, False])

        initial_states = {inst.id: (inst.priority, inst.confidence) for inst in self.instances}

        for inst in selected:
            self.update_priority_and_confidence(inst, is_useful)

        results = []
        for inst in self.instances:
            initial_priority, initial_confidence = initial_states[inst.id]
            if inst in selected:
                results.append([
                    inst.id,
                    initial_priority,
                    initial_confidence,
                    is_useful,
                    inst.priority,
                    inst.confidence
                ])
            else:
                results.append([
                    inst.id,
                    initial_priority,
                    initial_confidence,
                    None,
                    initial_priority,
                    initial_confidence
                ])

        decision = self.weighted_decision(selected)
        return selected, is_useful, results, decision

    def run_simulation(self, num_iterations=20):
        all_results = []
        for i in range(num_iterations):
            task = f"Task_{i}"
            selected, is_useful, results, decision = self.run_iteration(task)
            selected_ids = [inst.id for inst in selected]
            print(f"\nIteration {i+1}")
            print(f"Task: {task}")
            print(f"Selected instances: {selected_ids}")
            print(f"Is useful: {is_useful}")
            print(f"Weighted decision: {decision:.4f}")
            table = tabulate(results, headers=["ID", "Initial Priority", "Initial Confidence",
                                               "Is Useful", "Updated Priority", "Updated Confidence"],
                             floatfmt=".4f", tablefmt="grid")
            print(table)
            all_results.append(results)
        return all_results

system = FewShotSystem()
simulation_results = system.run_simulation()



Iteration 1
Task: Task_0
Selected instances: [1, 2, 3, 4, 9]
Is useful: True
Weighted decision: 3.8000
+------+--------------------+----------------------+-------------+--------------------+----------------------+
|   ID |   Initial Priority |   Initial Confidence | Is Useful   |   Updated Priority |   Updated Confidence |
+======+====================+======================+=============+====================+======================+
|    0 |             0.5000 |               0.5000 |             |             0.5000 |               0.5000 |
+------+--------------------+----------------------+-------------+--------------------+----------------------+
|    1 |             0.5000 |               0.5000 | True        |             0.5500 |               0.6000 |
+------+--------------------+----------------------+-------------+--------------------+----------------------+
|    2 |             0.5000 |               0.5000 | True        |             0.5500 |               0.6000 |
+------+

Gradient

In [ ]:
import random
import math
import numpy as np
from tabulate import tabulate

class Instance:
    def __init__(self, id, content):
        self.id = id
        self.content = content
        self.priority = 0.5  # Initial priority
        self.gradient = 0.0  # initial gradient
        self.use_count = 0

class FewShotSystem:
    def __init__(self, num_instances=20, instances_to_select=5, similarity_threshold=0.5, epsilon=0.2):
        self.instances = [Instance(i, f"Content_{i}") for i in range(num_instances)]
        self.instances_to_select = instances_to_select
        self.similarity_threshold = similarity_threshold
        self.epsilon = epsilon
        self.alpha = 0.1  # Attenuation parameters for positive feedback
        self.beta = 0.1  # Negative feedback attenuation parameter
        self.delta_p_positive = 0.05  # Initial positive adjustment
        self.delta_p_negative = 0.05  # initial negative adjustment amount

    def simulate_similarity(self, task, instance):
        return random.random()

    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def select_instances(self, task):
        similar_instances = [inst for inst in self.instances if self.simulate_similarity(task, inst) > self.similarity_threshold]
        if len(similar_instances) < self.instances_to_select:
            similar_instances = self.instances

        if random.random() < self.epsilon:
            return random.sample(similar_instances, self.instances_to_select)
        else:
            priorities = np.array([inst.priority for inst in similar_instances])
            normalized_priorities = self.softmax(priorities)
            sorted_instances = sorted(zip(similar_instances, normalized_priorities), key=lambda x: x[1], reverse=True)
            top_instances = [inst[0] for inst in sorted_instances[:self.instances_to_select]]
            return top_instances

    def update_priority_and_gradient(self, instance, is_useful):
        if is_useful:
            gradient_update = self.delta_p_positive * math.exp(-self.alpha * instance.use_count)
            instance.gradient = min(1.0, instance.gradient + gradient_update)
        else:
            gradient_update = self.delta_p_negative * math.exp(-self.beta * instance.use_count)
            instance.gradient = max(-1.0, instance.gradient - gradient_update)

        # Update priority based on gradient
        instance.priority = min(max(0.1, instance.priority + instance.gradient), 1.0)
        instance.use_count += 1

    def weighted_decision(self, selected_instances):
        total_weight = sum(inst.priority for inst in selected_instances)
        decision = sum(inst.priority * inst.id for inst in selected_instances) / total_weight if total_weight != 0 else 0
        return decision

    def run_iteration(self, task):
        selected = self.select_instances(task)
        is_useful = random.choice([True, False])

        initial_states = {inst.id: (inst.priority, inst.gradient) for inst in self.instances}

        for inst in selected:
            self.update_priority_and_gradient(inst, is_useful)

        results = []
        for inst in self.instances:
            initial_priority, initial_gradient = initial_states[inst.id]
            if inst in selected:
                results.append([
                    inst.id,
                    initial_priority,
                    initial_gradient,
                    is_useful,
                    inst.priority,
                    inst.gradient
                ])
            else:
                results.append([
                    inst.id,
                    initial_priority,
                    initial_gradient,
                    None,
                    initial_priority,
                    initial_gradient
                ])

        decision = self.weighted_decision(selected)
        return selected, is_useful, results, decision

    def run_simulation(self, num_iterations=20):
        all_results = []
        for i in range(num_iterations):
            task = f"Task_{i}"
            selected, is_useful, results, decision = self.run_iteration(task)
            selected_ids = [inst.id for inst in selected]
            print(f"\nIteration {i+1}")
            print(f"Task: {task}")
            print(f"Selected instances: {selected_ids}")
            print(f"Is useful: {is_useful}")
            print(f"Weighted decision: {decision:.4f}")
            table = tabulate(results, headers=["ID", "Initial Priority", "Initial Gradient",
                                               "Is Useful", "Updated Priority", "Updated Gradient"],
                             floatfmt=".4f", tablefmt="grid")
            print(table)
            all_results.append(results)
        return all_results

system = FewShotSystem()
simulation_results = system.run_simulation()


: 